In [1]:
import os
import s3fs
import pandas as pd

os.chdir("../")

from src.core.configs import settings

In [2]:
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': settings.s3_endpointUrl},
    key =settings.s3_key,
    secret = settings.s3_secret,
    token = settings.s3_token
)

In [7]:
pd.read_csv(fs.open(f"{settings.s3_prefix}_classes.csv"))

,filename,with_mask,without_mask
0,26-with-mask_jpg.rf.fb2a4fd14461ba4e7d981f51d9...,1,0
1,66_jpg.rf.fb48333563aba0cd06afba343f625d98.jpg,0,1
2,153-with-mask_jpg.rf.fb584ce29ce6d9a3123a04d80...,1,0
3,29-with-mask_jpg.rf.fb9ecac8d3dc335a43b79a9438...,1,0
4,augmented_image_315_jpg.rf.fba9c25c6675c5a039f...,1,0
...,...,...,...
1274,417_jpg.rf.64bed386d653f324066a9377a9dbbff8.jpg,0,1
1275,augmented_image_117_jpg.rf.6433692ef34560a302c...,0,1
1276,290_jpg.rf.65fd4f514fa0882844857ee5b4ff4880.jpg,0,1
1277,305-with-mask_jpg.rf.65c190086fbfb468313020747...,1,0
